## JSONL dosyasını açma

In [131]:
import jsonlines

veri = []
with jsonlines.open('all.jsonl') as reader:
    for obj in reader:
        veri.append(obj)


## Metni ve etiketleri işleme

In [132]:
conll_metni = ""

for obj in veri:
	text = obj['text']
	labels = obj['label']
	for baslangic, bitis, etiket in labels:
		kelimeler = text[baslangic:bitis].split()  # Etiketlenen kelimeyi ayır
		for kelime in kelimeler:
			conll_metni += kelime + "/" + etiket + " "  # Her kelimeyi etiketle birleştir
		conll_metni += "\n"  # Her cümleyi bir alt satıra yaz

print(conll_metni[:500])


Her/ANAT iki/ANAT memenin/ANAT 
cilt/ANAT yüzeyi/ANAT 
normal/OBS-PRESENT 
kalınlaşma/OBS-ABSENT 
retraksiyon/OBS-ABSENT 
Her/ANAT iki/ANAT meme/ANAT 
liposkleroze/OBS-PRESENT 
paternde/OBS-PRESENT 
Her/ANAT iki/ANAT meme/ANAT 
belirgin/OBS-PRESENT dens/OBS-PRESENT 
yapıdadır/OBS-PRESENT 
Sol/ANAT memede/ANAT 
alt/ANAT dış/ANAT kadranda/ANAT 
lineer/ANAT dizilimli/ANAT 
ince/OBS-PRESENT çizgisel/OBS-PRESENT 
kalsifikasyonlar/OBS-PRESENT 
Her/ANAT iki/ANAT aksillada/ANAT 
patolojik/OBS-ABSENT len


## İşlenen metini CoNLL formatına dönüştürme

In [140]:
conll_metni = ""

for obj in veri:
    text = obj['text']
    labels = obj['label']
    current_pos = 0
    for baslangic, bitis, etiket in labels:
        # Etiketlenmiş kelime arasındaki kelimeleri işle
        for kelime in text[current_pos:baslangic].split():
            conll_metni += kelime + "/O "  # Etiketsiz kelimeye "O" etiketi ekle
        # Etiketlenmiş kelimeyi işle
        etiketlenmis_kelime = text[baslangic:bitis]
        conll_metni += etiketlenmis_kelime + "/" + etiket + " "
        current_pos = bitis
    # Etiketlenmiş kelimenin sonrasındaki kelimeleri işle
    for kelime in text[current_pos:].split():
        conll_metni += kelime + "/O "
    conll_metni += "\n"

# for obj in veri:
#     text = obj['text']
#     labels = obj['label']
#     current_pos = 0
#     for baslangic, bitis, etiket in labels:
#         # Etiketlenmiş kelime arasındaki kelimeleri işle
#         for kelime in text[current_pos:baslangic].split():
#             conll_metni += kelime + "/O "  # Etiketsiz kelimeye "O" etiketi ekle
#         # Etiketlenmiş kelimeyi işle ve kelimeleri ayır
#         etiketlenmis_kelime = text[baslangic:bitis]
#         kelimeler = etiketlenmis_kelime.split()
#         for kelime in kelimeler:
#             conll_metni += kelime + "/" + etiket + " "  # Her kelimeyi etiketle birleştir
#         current_pos = bitis
#     # Etiketlenmiş kelimenin sonrasındaki kelimeleri işle
#     for kelime in text[current_pos:].split():
#         conll_metni += kelime + "/O "
#     conll_metni += "\n"

with open("veri.conll", "w", encoding="utf-8") as f:
    f.write(conll_metni)

print(conll_metni[:500])


BİLATERAL/O MAMOGRAFİ/O İNCELEMESİ:/O Her iki memenin/ANAT cilt yüzeyi/ANAT normal/OBS-PRESENT olarak/O değerlendirilmiş/O olup/O kalınlaşma/OBS-ABSENT ve/O retraksiyon/OBS-ABSENT saptanmamıştır./O Her iki meme/ANAT liposkleroze /OBS-PRESENT paternde/OBS-PRESENT izlenmektedir./O Her iki meme/ANAT belirgin dens/OBS-PRESENT yapıdadır/OBS-PRESENT ./O Sol memede/ANAT alt dış kadranda/ANAT  lineer dizilimli /ANAT ince çizgisel /OBS-PRESENT kalsifikasyonlar/OBS-PRESENT izlendi./O Her iki aksillada/ANA


## SpaCy modelini oluşturma

In [134]:
import spacy
from spacy.training.example import Example

# SpaCy projesi oluşturma
nlp = spacy.blank("tr")
ner = nlp.add_pipe("ner", name="ner")



## Eğitim verisini yükleme

In [135]:
with open("veri.conll", "r") as f:
    train_data = f.readlines()

print(train_data[:5])  # İlk 5 satırı yazdır

['BİLATERAL/O MAMOGRAFİ/O İNCELEMESİ:/O Her/ANAT iki/ANAT memenin/ANAT cilt/ANAT yüzeyi/ANAT normal/OBS-PRESENT olarak/O değerlendirilmiş/O olup/O kalınlaşma/OBS-ABSENT ve/O retraksiyon/OBS-ABSENT saptanmamıştır./O Her/ANAT iki/ANAT meme/ANAT liposkleroze/OBS-PRESENT paternde/OBS-PRESENT izlenmektedir./O Her/ANAT iki/ANAT meme/ANAT belirgin/OBS-PRESENT dens/OBS-PRESENT yapıdadır/OBS-PRESENT ./O Sol/ANAT memede/ANAT alt/ANAT dış/ANAT kadranda/ANAT lineer/ANAT dizilimli/ANAT ince/OBS-PRESENT çizgisel/OBS-PRESENT kalsifikasyonlar/OBS-PRESENT izlendi./O Her/ANAT iki/ANAT aksillada/ANAT patolojik/OBS-ABSENT lenf/OBS-ABSENT bezi/OBS-ABSENT izlenmemektedir./O SONUÇ:/O BIRADS/IMPRESSION 5/IMPRESSION \n', 'BİLATERAL/O MAMOGRAFİ/O İNCELEMESİNDE;/O Bilateral/ANAT meme/ANAT cilt/ANAT kalınlığı/ANAT normaldir/OBS-PRESENT ./O Her/ANAT iki/ANAT meme/ANAT tip/OBS-PRESENT b/OBS-PRESENT paterndedir/OBS-PRESENT ./O Her/ANAT iki/ANAT memede/ANAT spiküle/OBS-ABSENT konturlu/OBS-ABSENT kitle/OBS-ABSENT lezy

## CoNLL formatındaki veriyi Example formatına dönüştürme

In [136]:
train_examples = []

for line in train_data:
    words_and_labels = line.strip().split()

    # Veriyi doğrulama
    if len(words_and_labels) < 2:  # Kelime ve etiket olmalı
        continue

    words = [word_label.split("/")[0] for word_label in words_and_labels]
    labels = [word_label.split("/")[1] if len(word_label.split("/")) > 1 else "O" for word_label in words_and_labels]  # "O" etiketi alınması için kontrol ekle
    train_examples.append(Example.from_dict(nlp.make_doc(" ".join(words)), {"entities": [(i, i+1, labels[i]) for i in range(len(labels))]}))

with open("a.txt", "w", encoding="utf-8") as f:
    for example in train_examples:
        f.write(str(example) + "\n")

print(train_examples[:5])  # İlk 5 örneği yazdır


[{'doc_annotation': {'cats': {}, 'entities': ['O', 'O', 'O', 'U-ANAT', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], 'spans': {}, 'links': {}}, 'token_annotation': {'ORTH': ['BİLATERAL', 'MAMOGRAFİ', 'İNCELEMESİ', ':', 'Her', 'iki', 'memenin', 'cilt', 'yüzeyi', 'normal', 'olarak', 'değerlendirilmiş', 'olup', 'kalınlaşma', 've', 'retraksiyon', 'saptanmamıştır', '.', 'Her', 'iki', 'meme', 'liposkleroze', 'paternde', 'izlenmektedir', '.', 'Her', 'iki', 'meme', 'belirgin', 'dens', 'yapıdadır', '.', 'Sol', 'memede', 'alt', 'dış', 'kadranda', 'lineer', 'dizilimli', 'ince', 'çizgisel', 'kalsifikasyonlar', 'izlendi', '.', 'Her', 'iki', 'aksillada', 'patolojik', 'lenf', 'bezi', 'izlenmemektedir', '.', 'SONUÇ', ':', 'BIRADS', '5'], 'SPACY': [True, True, False, True, True, True, True, 

## Modeli eğitme

In [137]:
# Modeli eğitme
nlp.begin_training()
for i in range(20):  # Örneğin 10 epoch için eğitim
    nlp.update(train_examples)

# Modeli kaydetme
nlp.to_disk("ner_model")

# TEST

## Modeli yükleme

In [138]:
import spacy

nlp = spacy.load("ner_model")


## Modeli test etme

In [139]:

test_data = [
    {"text": "BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup kalınlaşma ve retraksiyon saptanmamıştır. \nHer iki meme liposkleroze paternde izlenmektedir. \nHer iki meme belirgin dens yapıdadır.\nSol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.\nHer iki aksillada patolojik lenf bezi izlenmemektedir.\nSONUÇ: BIRADS 5"},
]

# Test verisini işleme ve modelin sonuçlarını alma
for obj in test_data:
    text = obj['text']
    doc = nlp(text)
    print("Metin:", text)
    print("Modelin Tahmin Ettiği Etiketler:")
    for ent in doc.ents:
        print(ent.text, ent.label_)


Metin: BİLATERAL MAMOGRAFİ İNCELEMESİ:
Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup kalınlaşma ve retraksiyon saptanmamıştır. 
Her iki meme liposkleroze paternde izlenmektedir. 
Her iki meme belirgin dens yapıdadır.
Sol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.
Her iki aksillada patolojik lenf bezi izlenmemektedir.
SONUÇ: BIRADS 5
Modelin Tahmin Ettiği Etiketler:
:
 OBS-PRESENT


In [145]:
import spacy
from spacy.training import Example

# Örnek veriyi yükleme
data = """BİLATERAL/O MAMOGRAFİ/O İNCELEMESİ:/O Her iki memenin/ANAT cilt yüzeyi/ANAT normal/OBS-PRESENT olarak/O değerlendirilmiş/O olup/O kalınlaşma/OBS-ABSENT ve/O retraksiyon/OBS-ABSENT saptanmamıştır./O Her iki meme/ANAT liposkleroze/OBS-PRESENT paternde/OBS-PRESENT izlenmektedir./O Her iki meme/ANAT belirgin dens/OBS-PRESENT yapıdadır/OBS-PRESENT ./O Sol memede/ANAT alt dış kadranda/ANAT lineer dizilimli/ANAT ince çizgisel/OBS-PRESENT kalsifikasyonlar/OBS-PRESENT izlendi./O Her iki aksillada/ANAT patolojik lenf bezi/OBS-ABSENT izlenmemektedir./O SONUÇ:/O BIRADS 5/O"""

# Veriyi kelimelere ve etiketlere ayırma
def parse_data(data):
    sentences = []
    labels = []
    for sentence in data.split("\n"):
        words_labels = sentence.split()
        words = []
        entities = []
        for word_label in words_labels:
            if "/" in word_label:
                word, label = word_label.rsplit("/", 1)
                words.append(word)
                entities.append(label)
            else:
                words.append(word_label)
                entities.append("O")
        sentences.append(words)
        labels.append(entities)
    return sentences, labels

sentences, labels = parse_data(data)

# SpaCy modelini oluşturma
nlp = spacy.blank("tr")  # Türkçe dil modeli
ner = nlp.add_pipe("ner", last=True)

# Verileri SpaCy formatına dönüştürme
def convert_to_spacy_format(sentences, labels):
    spacy_data = []
    for sentence, ents in zip(sentences, labels):
        doc = nlp.make_doc(" ".join(sentence))
        entities = []
        start_idx = 0
        for i, word in enumerate(sentence):
            start_idx = doc.text.find(word, start_idx)
            end_idx = start_idx + len(word)
            if ents[i] != 'O':
                entities.append((start_idx, end_idx, ents[i]))
            start_idx = end_idx + 1
        spacy_data.append(Example.from_dict(doc, {"entities": entities}))
    return spacy_data

train_data = convert_to_spacy_format(sentences, labels)

# Modeli eğitme
optimizer = nlp.begin_training()
for i in range(258):  # 10 epoch boyunca eğit
    losses = {}
    for batch in spacy.util.minibatch(train_data, size=2):
        nlp.update(batch, drop=0.5, losses=losses)
    print(f"Losses at iteration {i}: {losses}")

# Modeli kaydetme
nlp.to_disk("ner_model")

# Modeli test etme
nlp = spacy.load("ner_model")
doc = nlp("Test cümlenizi buraya yazın")
for ent in doc.ents:
    print(ent.text, ent.label_)


Losses at iteration 0: {'ner': 51.99999564886093}
Losses at iteration 1: {'ner': 50.96921896934509}
Losses at iteration 2: {'ner': 50.55144011974335}
Losses at iteration 3: {'ner': 48.323278188705444}
Losses at iteration 4: {'ner': 47.05524271726608}
Losses at iteration 5: {'ner': 45.209733724594116}
Losses at iteration 6: {'ner': 43.49134171009064}
Losses at iteration 7: {'ner': 41.72501593828201}
Losses at iteration 8: {'ner': 39.92473262548447}
Losses at iteration 9: {'ner': 38.29779249429703}
Losses at iteration 10: {'ner': 35.33222311735153}
Losses at iteration 11: {'ner': 33.9009545147419}
Losses at iteration 12: {'ner': 30.92587199807167}
Losses at iteration 13: {'ner': 28.868030041456223}
Losses at iteration 14: {'ner': 28.1363513097167}
Losses at iteration 15: {'ner': 29.09907630458474}
Losses at iteration 16: {'ner': 26.532006414607167}
Losses at iteration 17: {'ner': 27.558814618736506}
Losses at iteration 18: {'ner': 26.639113882556558}
Losses at iteration 19: {'ner': 26.23

In [151]:
import spacy

nlp = spacy.load("ner_model")

test_data = [
    {"text": "BİLATERAL MAMOGRAFİ İNCELEMESİ:\nHer iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup kalınlaşma ve retraksiyon saptanmamıştır. \nHer iki meme liposkleroze paternde izlenmektedir. \nHer iki meme belirgin dens yapıdadır.\nSol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.\nHer iki aksillada patolojik lenf bezi izlenmemektedir.\nSONUÇ: BIRADS 5"},
]

# Test verisini işleme ve modelin sonuçlarını alma
for obj in test_data:
    text = obj['text']
    doc = nlp(text)
    print("\nMetin:\n\n", text)
    print("\n\nModelin Tahmin Ettiği Etiketler:\n")
    for ent in doc.ents:
        print(ent.text, ent.label_)



Metin:

 BİLATERAL MAMOGRAFİ İNCELEMESİ:
Her iki memenin cilt yüzeyi normal olarak değerlendirilmiş olup kalınlaşma ve retraksiyon saptanmamıştır. 
Her iki meme liposkleroze paternde izlenmektedir. 
Her iki meme belirgin dens yapıdadır.
Sol memede alt dış kadranda lineer dizilimli ince çizgisel kalsifikasyonlar izlendi.
Her iki aksillada patolojik lenf bezi izlenmemektedir.
SONUÇ: BIRADS 5


Modelin Tahmin Ettiği Etiketler:

memenin ANAT
yüzeyi ANAT
normal OBS-PRESENT
kalınlaşma OBS-ABSENT
retraksiyon OBS-ABSENT
meme ANAT
liposkleroze OBS-PRESENT
paternde OBS-PRESENT
meme ANAT
dens OBS-PRESENT
yapıdadır OBS-PRESENT
memede ANAT
kadranda ANAT
dizilimli ANAT
çizgisel OBS-PRESENT
kalsifikasyonlar OBS-PRESENT
aksillada ANAT
bezi OBS-ABSENT
